# DataNights-VISceral
#### Código estructurado para las #DataNights de VISceral

<details>
    <summary><strong>Meta</strong></summary>
    La meta de este notebook es recrear el análisis de <a href="https://twitter.com/moaimx">@moaimx</a> que hizo en <a href="https://twitter.com/search?q=%23datanights&src=typed_query">#DataNights</a>
    <ul>
        <li> Limpiar las bases de datos del Secretariado sobre carpetas de investigación y víctimas de homicidios dolosos. </li>
        <ul>
            <li> Extraer y transformar el conjunto de datos <strong>Estatal-Delitos-2015-2019_mar19.zip</strong> y <strong>Estatal-Víctimas-2015-2019_mar19.zip</strong></li>
            <li> Guardar las bases de datos en formato .csv para facilitar el uso en un futuro. </li>
            <li> Crear una base de datos lista para visualización (en formato <i>tidy</i>). </li>
        </ul>
    </ul>
</details>

<details>
    <summary><strong>Contexto</strong></summary>
    Adquirimos los datos en bruto del <a href="http://www.morlan.mx/blog/2019/04/21/datanights-y-el-regreso-de-este-blog/">blog sobre #DataNights </a> en <a href="morlan.mx">morlan.mx</a>
    También seguiremos cercanamente los pasos para recrear lo que hicieron en los primeros dos episodios:
    <ul>
        <li><a href="https://www.youtube.com/watch?v=K2cYa41zmdw"> Episodio 1 </a></li>
        <li><a href="https://www.youtube.com/watch?v=xxQlCioKTe8"> Episodio 2 </a></li>
    </ul>
</details>

In [ ]:
import pandas as pd
from zipfile import ZipFile
from pathlib import Path
from herramientas import arbol
from datetime import datetime as dt
hoy = dt.today().strftime("%d-%b-%y")

hoy

In [ ]:
DATOS_BRUTOS = Path("../datos/brutos/")
DATOS_INTERINOS = Path("../datos/interinos/")
DATOS_PROCESADOS = Path("../datos/procesados/")
DATOS_EXTERNOS = Path("../datos/externos/")

In [ ]:
arbol(DATOS_BRUTOS)

***
## Extraer datos de un archivo .zip
Para extraer datos de un archivo `.zip` utilizamos la biblioteca 📚 de python `zipfile` de la siguiente manera:

<details>
    <summary>Nota</summary>
    En la primera celda de código de este notebook importamos el objeto <code style="background-color:#eeeeee">ZipFile</code> de la biblioteca <code style="background-color:#eee">zipfile</code>
</details>

In [ ]:
archivo_zip_delitos = ZipFile(DATOS_BRUTOS / "Estatal-Delitos-2015-2019_mar19.zip")

<details>
    <summary>Nota sobre <code style="background-color:#eee">DATOS_BRUTOS / <span style="color:#ba2121">"Estatal-Delitos-2015-2019_mar19.zip"</span></code></summary>
    El objeto <code style="background-color:#eeeeee">Path</code> de la biblioteca <code style="background-color:#eee">pathlib</code> tiene la "habilidad" de conectar rutas de archivos utilizando el símbolo "/". Independientemente del sistema operativo que utilices. <br>
    Lo cual significa que si <code style="background-color:#eee">DATOS_BRUTOS / <span style="color:#ba2121">"Estatal-Delitos-2015-2019_mar19.zip"</span></code> es igual a <code style="background-color:#eee;color:#ba2121">"../datos/brutos/Estatal-Delitos-2015-2019_mar19.zip"</code> <br>
    o <code style="background-color:#eee;color:#ba2121">"..\datos\brutos\Estatal-Delitos-2015-2019_mar19.zip"</code> si estas en Windows (se formatea automaticamente).
</details>

Ahora `archivo_zip_delitos` es un objeto `ZipFile` así que podemos hacer lo siguiente:

In [ ]:
# ver la lista de archivos que tiene
archivo_zip_delitos.filelist

extraer los archivos

In [ ]:
archivo_zip_delitos.extractall(path = DATOS_INTERINOS)

Ahora en nuestro archivo `datos/interinos/` existe el archivo `'Estatal Delitos - marzo 2019.xlsb'`

In [ ]:
arbol(DATOS_INTERINOS)

Hagamos lo mismo con el archivo de víctimas.

In [ ]:
archivo_zip_victimas = ZipFile(DATOS_BRUTOS / 'Estatal-Víctimas-2015-2019_mar19.zip')
archivo_zip_victimas.extractall(DATOS_INTERINOS)

In [ ]:
arbol(DATOS_INTERINOS)

***
## Leer archivos Excel en `Pandas`

`Pandas` tiene la habilidad de leer archivos excel directamente pero un archivo excel esta compuesto de varias hojas así que leemos estas hojas en dos pasos:

In [ ]:
archivo_delitos = pd.ExcelFile(DATOS_INTERINOS / 'Estatal Delitos - marzo 2019.xlsb', )

Archivos XLSB (Excel Binario) no tienen soporte en las bibliotecas modernas de python pero existe una biblioteca no muy conocida llamada `pyxlsb` que nos puede ayudar. <br>
La puedes instalar con `pip` desde tu terminal:
```shell
pip install pyxlsb
```

In [ ]:
import pyxlsb
# vamos a excribir csv a la antiüita
import csv

In [ ]:
with pyxlsb.open_workbook(DATOS_INTERINOS / 'Estatal Delitos - marzo 2019.xlsb') as workbook:
    for nombre in workbook.sheets:
        with workbook.get_sheet(nombre) as hoja, open(DATOS_INTERINOS / f"delitos-{nombre}.csv", 'w',  encoding = 'utf-8') as f:
            escritor_csv = csv.writer(f)
            for fila in hoja.rows():
                escritor_csv.writerow([celda.v for celda in fila])

In [ ]:
arbol(DATOS_INTERINOS)

Lo que estamos haciendo es:
1. Abrir el archivo y asignandolo a la variable "workbook"
2. Creamos una "loop" (o bucle) para hacer X ejercicio en cada hoja del workbook (`workbook.sheets`)
3. Creamos un archivo "delitos-{nombre de la hoja}.csv" 
4. Creamos un "escritor" de csv 
5. Hacemos otra "loop" (o bucle) y por cada fila en nuestra hoja creamos una lista con los valores de cada celda en esa fila la cual escribimos a nuestro archivo .csv <- esta es la parte más díficil de comprender porque son 3 pasos en uno.

Hagamos lo mismo con el archivo de víctimas:

In [ ]:
with pyxlsb.open_workbook(DATOS_INTERINOS / 'Estatal Víctimas - marzo 2019.xlsb') as workbook:
    for nombre in workbook.sheets:
        with workbook.get_sheet(nombre) as hoja, open(DATOS_INTERINOS / f"victimas-{nombre}.csv", 'w', encoding = 'utf-8') as f:
            escritor_csv = csv.writer(f)
            for fila in hoja.rows():
                escritor_csv.writerow([celda.v for celda in fila])

In [ ]:
arbol(DATOS_INTERINOS)

Esta "loop" crea un archivo `.csv` por cada hoja en el archivo `.xlsb` así que este pedacito de código funciona aunque tu archivo excel tenga 150 hojas. 

***
## Creando un conjunto de datos listos para visualización (en formato `tidy`)

Hasta el momento llevamos 2 de nuestras 3 metas:<br>
<input type="checkbox" checked> Extraer y transformar el conjunto de datos <strong>Estatal-Delitos-2015-2019_mar19.zip</strong> y <strong>Estatal-Víctimas-2015-2019_mar19.zip</strong><br>
<input type="checkbox" checked> Guardar las bases de datos en formato .csv para facilitar el uso en un futuro.<br>
<input type="checkbox" > Crear una base de datos lista para visualización (en formato <i>tidy</i>).

El limpiar y restructurar datos siempre es lo que lleva más tiempo en el proceso de análisis de datos.

In [ ]:
delitos = pd.read_csv(DATOS_INTERINOS / 'delitos-Hoja1.csv')
victimas = pd.read_csv(DATOS_INTERINOS / 'victimas-Hoja1.csv')

In [ ]:
delitos.head()

In [ ]:
victimas.head()

#### NOTA:
Este tutorial **asume** que viste los episodios 1 y 2 de #DataNights. No vamos a explicar porque estamos escogiendo las variables que estamos escogiendo o lo que ilustran cada uno de estos conjuntos de datos.

El producto final de ambos conjuntos (víctimas y delitos) es parecido: un conjunto de datos con las columnas "Año", "Mes", "Delitos" o "Víctimas" donde esta la cuenta total (agregada) de los delitos o víctimas en México en ese Año-Mes.

### Filtrando datos
Primero vemos los valores _unicos_ en nuestra columna 'Supbipo de delito' para asegurarnos de que no haya errores de dedo. Por ejemplo, en bases de datos de baja calidad puedes encontrar:
1. "Homicidio doloso"
2. "Homicidio doloso "
3. " Homicidio doloso"
4. "homicidio doloso"
5. "Homicidio Doloso"

Todos estos valores son distintos para `pandas`. 

In [ ]:
delitos['Subtipo de delito'].unique()

Ya que vemos que nuestra columna 'Subtipo de delito' si está bien construida/limpia podemos crear "máscaras" para filtrar los datos.

In [ ]:
mascara_homicidio_doloso_delitos = delitos['Subtipo de delito'] == 'Homicidio doloso'
mascara_homicidio_doloso_victimas = victimas['Subtipo de delito'] == 'Homicidio doloso'

Para más información sobre como funciona esto ve los videos de DataNights.

In [ ]:
delitos[mascara_homicidio_doloso_delitos].head()

In [ ]:
victimas[mascara_homicidio_doloso_victimas].head()

Nota el _brinco_ en el índice en ambos **DataFrames**. Esto significa que hemos eliminado todas las filas que no cumplan el requisito de la "máscara" (que la columna "Subtipo de delito" equivalga "Homicidio doloso").

In [ ]:
delitos = delitos[mascara_homicidio_doloso_delitos].copy()
victimas = victimas[mascara_homicidio_doloso_victimas].copy()

<details>
    <summary>Nota sobre <code style="background-color:#eee">.<span style="color:#0055AA">copy</span>()</code></summary>
    Utilizar <code style="background-color:#eee">.<span style="color:#0055AA">copy</span>()</code> no es necesario ya que estamos sobreescribiendo la variable delitos. <code style="background-color:#eee">.<span style="color:#0055AA">copy</span>()</code> es necesario cuando estas creando una copia de una parte de otro DataFrame. Pandas por defecto va a crear una especie de "vista" en lugar de una copia copia. Esto puede tener consecuencias no deseadas más adelante en tus análisis. Así que es buena practica utilizar <code style="background-color:#eee">.<span style="color:#0055AA">copy</span>()</code>
</details>

### Trabajando con fechas
`python` es muy bueno trabajando con fechas. En la biblioteca estándard (que viene incluida en `python`) existe el paquete `datetime` para trabajar con fechas y tiempos. Es posible utilizar este paquete para trabajar con fechas en español también (sólo es cuestión de cambiar el `LC_TIME`) pero existen otros paquetes como `arrow` que hacen el proceso de trabajar con fechas mucho más amigable.

In [ ]:
import arrow

In [ ]:
arrow.locales.SpanishLocale.month_names

`Arrow` tiene "locales" para muchos muchos idiomas. En este caso vamos a utilizar el `SpanishLocale` para obtener una lista de los meses más rapidamente que escribiendolos a mano.

In [ ]:
# esta es una list comprehension - ve el video de DataNights para aprender más sobre esto
meses = [mes.title() for mes in arrow.locales.SpanishLocale.month_names if mes]
meses

Nota: agregamos el `if mes` para decirle a `python` que me de un `mes` con la primera letra en mayúscula (`.title()`) **sólo** si `mes` existe/es `True`. En `python` las cadenas ( o _strings_) tienen un valor equivalente a `False` si están vacías (como el primer valor de la lista `arrow.locales.SpanishLocale.month_names`).

Ahora podemos crear una lista de "columnas de interés", las columnas que vamos a utilizar en nuestro análisis. Aquí es de donde partimos un poco del proceso utilizado en DataNights. 

Hacemos esto por preferencia personal. Antes de restructurar conjuntos de datos, a mí, en lo personal, me gusta filtrar todo lo que no voy a utilizar. Esto significa, eliminar filas **y** columnas que no crea necesarias para mi análisis. 

Primero creamos una lista de columnas que nos interesan. En este caso van a ser el Año y los meses. Como ya tenemos una lista de meses lo que hacemos es `extend`er una primera lista (que solo tiene "Año") para incluir los meses.

In [ ]:
columnas_de_interes = ['Año']
columnas_de_interes.extend(meses)
columnas_de_interes

Esto funciona igual que las máscaras que utilizamos anteriormente para filtrar filas. Y ya que ambos **DataFrame**s tienen las mismas columnas, no ocupamos crear dos "máscaras".

In [ ]:
delitos[columnas_de_interes].head()

In [ ]:
victimas[columnas_de_interes].head()

In [ ]:
delitos = delitos[columnas_de_interes].copy()
victimas = victimas[columnas_de_interes].copy()

### Derretir nuestos datasets
Reitero: No vamos a explicar todo esto en este tutorial. Si no sabes lo que estamos haciendo aquí, ve a ver los episodios 1 y 2 de #DataNights.

In [ ]:
delitos_derretido = delitos.melt(id_vars = 'Año', var_name = 'Mes', value_name = 'Cuenta (carpetas)')
victimas_derretido = victimas.melt(id_vars = 'Año', var_name = 'Mes', value_name = 'Cuenta (victimas)')

Ya que tenemos nuestros nuevos **DataFrame**s en esta nueva estructura (derretida) es cuando yo modificaría los valores.

In [ ]:
victimas_derretido.head()

Necesitamos un _diccionario_ con los nombres del mes como llave y sus valores correspondientes (01-12). 

Igual que hicimos una "list comprehension" podemos hacer un "dict comprehension"

In [ ]:
{mes: valor for valor, mes in enumerate(meses, start = 1)}

Pero queremos los valores en formato "01" en lugar de 1. Para eso utilizamos `.zfill()`

In [ ]:
{mes: str(valor).zfill(2) for valor, mes in enumerate(meses, start = 1)}

In [ ]:
sustituciones_meses = {mes: str(valor).zfill(2) for valor, mes in enumerate(meses, start = 1)}

Ahora podemos sustituir nuestra columna de "Mes" por valores númericos (más o menos númericos).

In [ ]:
victimas_derretido["Mes"] = victimas_derretido["Mes"].map(sustituciones_meses)
delitos_derretido["Mes"] = delitos_derretido["Mes"].map(sustituciones_meses)

In [ ]:
victimas_derretido.head()

Ahora transformamos nuestra columna "Año" a _string_ para concatenarla con "Mes" para tener una columna "Año-Mes" que utilizaremos como nuestro eje X.

Primero la convertimos a `int` o enteros para deshacernos del .0 al final de cada número.

In [ ]:
victimas_derretido['Año'] = victimas_derretido['Año'].astype(int).astype(str)
delitos_derretido['Año'] = delitos_derretido['Año'].astype(int).astype(str)

In [ ]:
victimas_derretido.head()

In [ ]:
victimas_derretido['Fecha'] = victimas_derretido["Año"] + '-' + victimas_derretido['Mes']
delitos_derretido['Fecha'] = delitos_derretido["Año"] + '-' + delitos_derretido['Mes']

In [ ]:
victimas_derretido.head()

Ya que tenemos estos valores "Fecha" podemos agrupar nuestros datos por esta columna y agregarlos:

In [ ]:
victimas_derretido.groupby("Fecha")['Cuenta (victimas)'].sum().head()

In [ ]:
victimas_derretido.groupby("Fecha")[['Cuenta (victimas)']].sum().head()

Nota que si utilizas [["___________"]] en lugar de un solo par de corchetes recibirás un **DataFrame** en lugar de una **Serie** de pandas por defecto.

In [ ]:
victimas_agrupado = victimas_derretido.groupby("Fecha")[['Cuenta (victimas)']].sum().reset_index()
delitos_agrupado = delitos_derretido.groupby("Fecha")[['Cuenta (carpetas)']].sum().reset_index()

In [ ]:
victimas_agrupado.head()

Pero como vimos en #DataNights, nuestro conjunto de datos tiene columnas para todos los meses del año pero valores solo hasta marzo 2019. Esto significa que empezando en abril, los valores son en nuestro conjunto modificado son 0:

In [ ]:
victimas_agrupado.tail()

Como nuestros datos estan ordenados podemos hacer algo como

In [ ]:
victimas_agrupado[:-9]

eliminando las ultimas 9 filas del **DataFrame**. También sabemos ningún par de Año-Mes tiene un valor equivalente a 0 de manera natural así que podemos crear una máscara para filtrar las filas que no cumplan este requisito. 

In [ ]:
mascara_cuenta_victimas = victimas_agrupado['Cuenta (victimas)'] > 0
mascara_cuenta_delitos = delitos_agrupado['Cuenta (carpetas)'] > 0

In [ ]:
delitos_agrupado[mascara_cuenta_delitos]

In [ ]:
delitos_agrupado = delitos_agrupado[mascara_cuenta_delitos].copy()
victimas_agrupado = victimas_agrupado[mascara_cuenta_victimas].copy()

# ¡Listo!
Tenemos nuestros conjuntos de datos listos para visualizar. 
Guardemoslos en formato `.csv` para facilitar su uso en un futuro.

In [ ]:
delitos_agrupado.to_csv(DATOS_PROCESADOS / f'carpetas-cuenta-{hoy}.csv', encoding = 'utf-8', index = False)
victimas_agrupado.to_csv(DATOS_PROCESADOS / f'victimas-cuenta-{hoy}.csv', encoding = 'utf-8', index = False)

***
## Visualización
Para mantener nuestro entorno de trabajo organizado crearemos nuestra visualización en otro notebook. 